# Assignment 28: Combining Pandas DataFrames #

### Goals for this Assignment ###

By the time you have completed this assignment, you should be able to:

- Use Pandas' `concat` function to combine two Pandas `DataFrame` objects
- Use the `merge` method on Pandas `DataFrame` objects to perform an _inner join_ with another `DataFrame`
- Use the `merge` method on Pandas `DataFrame` objects to perform an _outer join_ with another `DataFrame`

## Step 1: Use `concat` To Combine two Pandas `DataFrame` Objects ##

### Background: Combining Multiple `DataFrame`s ###

Sometimes we want to combine multiple `DataFrame` objects together into a single `DataFrame`.
For example, say a doctor's office tracks the last blood pressure reading from its patients.
Because of the sheer volume of patients, readings are divided into separate files, based on the first character of the patient's name.
For example, we might have one file associated with patients whose names start with "a":

| Patient Name | Age | Blood Pressure |
| ------------ | --- | -------------- |
| Abigail      | 45  | 135            |
| Adam         | 57  | 122            |
| Alice        | 25  | 118            |
| Andrew       | 62  | 121            |


...and another file associated with patients whose names start with "b":

| Patient Name | Age | Blood Pressure |
| ------------ | --- | -------------- |
| Barbara      | 30  | 132            |
| Bob          | 22  | 135            |
| Bill         | 43  | 126            |

While this split might make sense from a data storage perspective (e.g., reducing the size of any individual file), this probably does not make much sense from an analysis perspective.
That is, you're more likely to want to analyze all patients, not just the ones whose name starts with a given letter.
The bad news here is that you're forced into reading all these files separately, leading to a separate `DataFrame` object for each file.
This is not conducive to doing analyses over all patients.

The good news is that Pandas offers a number of ways to combine multiple `DataFrame` objects together.
One way to do this is via `concat`, which takes a list of `DataFrame` objects to concatenate together.
This is shown in the next cell, using the data from the above two tables as an example.

In [1]:
import pandas as pd

patients_a = pd.DataFrame({"name": ["abigail", "adam", "alice", "andrew"],
                           "age": [45, 57, 25, 62],
                           "blood_pressure": [135, 122, 118, 121]})
patients_b = pd.DataFrame({"name": ["barbara", "bob", "bill"],
                           "age": [30, 22, 43],
                           "blood_pressure": [132, 135, 126]})
patients_combined = pd.concat([patients_a, patients_b])
print(patients_combined)

      name  age  blood_pressure
0  abigail   45             135
1     adam   57             122
2    alice   25             118
3   andrew   62             121
0  barbara   30             132
1      bob   22             135
2     bill   43             126


If you run the above cell, you'll see all the data from both `DataFrame` objects put into the same output `DataFrame` object.
However, if you look closely, there is something strange: **multiple** rows have the same index!
This is not just some weird artifact of `print`ing this out, as shown in the next cell:

In [2]:
print(patients_combined["name"][0])

0    abigail
0    barbara
Name: name, dtype: object


If you run the above cell, you'll see that this gives you back a Pandas `Series` object, containing both `"abigail"` and `"barbara"`.
That is, _both_ are at row index `0`.
This is because `concat`, by default, maintains the original row indices from all of its input `DataFrame` objects.
This behavior makes sense given the knowledge that row indices can be user-controlled (and therefore given meaningful names), but in this case, we only are using numeric identifiers which are intended to be sequential.

We can address this situation by telling `concat` to ignore the original row indices, as shown in the following cell:

In [3]:
patients_combined = pd.concat([patients_a, patients_b], ignore_index=True)
print(patients_combined)

      name  age  blood_pressure
0  abigail   45             135
1     adam   57             122
2    alice   25             118
3   andrew   62             121
4  barbara   30             132
5      bob   22             135
6     bill   43             126


If the above cell is run, you'll see that now each row has its own unique index.
This is further demonstrated in the next cell where we access the same row index again:

In [4]:
print(patients_combined["name"][0])

abigail


If run, you'll see that this now gives us the name `"abigail"`, as opposed to a Pandas `Series` object containing names.

### Try this Yourself ###

In the next cell, there are two `DataFrame` objects `furniture_first` and `furniture_second` with the same columns representing an inventory of furniture.
Combine these two into a single `DataFrame` object `furniture_combined`, using `concat`.
Each row should have a unique index.

In [6]:
furniture_first = pd.DataFrame({"name": ["table", "chair", "couch"],
                                "price": [150, 80, 500]})
furniture_second = pd.DataFrame({"name": ["bed", "dresser"],
                                 "price": [600, 200]})

# write your code here which will create furniture_combined with the help
# of Pandas' concat
furniture_combined = pd.concat([furniture_first, furniture_second], ignore_index=True)

print(furniture_combined)
# above statement should print:
#       name  price
# 0    table    150
# 1    chair     80
# 2    couch    500
# 3      bed    600
# 4  dresser    200

      name  price
0    table    150
1    chair     80
2    couch    500
3      bed    600
4  dresser    200


## Step 2: Use `merge` To Combine two Pandas `DataFrame` Objects via an Inner Join ##

### Background: Inner Joins ###

Sometimes we have two closely-related `DataFrame` objects which share some, but not all columns.
For example, say we are representing the following information with two files at a hardware store:

1. A file containing inventory information.  This includes which items are available, their prices, their availabile quantities, and what locations the items are in.  Each individual item in inventory could be represented as a separate row.
2. A file containing customer purchase information.  This includes columns for who purchased what, and when.  Each individual item purchased could be represented as a separate row.

It would arguably not make sense to track all this information in the same file (or at least within the same table), given that there are columns which are not shared by both tables.
Nonetheless, there is _some_ overlap between columns, namely, specific items.
Moreover, there are situations in which an analysis might need information from both files.
For example, if we want to determine how much a given customer should have paid on a given day, we would need to cross-reference purchase information from the second file against price information in the first file.
For ease of analysis, it would make the most sense to somehow put this cross-referenced information into a single `DataFrame`.
This means that ultimately, we need a way to combine the `DataFrame` objects corresponding to the two files above into a single `DataFrame` object holding cross-referenced information.

While we saw that `concat` can be used to combine `DataFrame` objects, it is not well-suited to this sort of cross-referencing.
This is shown in the next cell.

In [7]:
hardware_inventory = pd.DataFrame({"product": ["hammer", "wrench", "screws (20)"],
                                   "price": [20, 30, 2.5],
                                   "quantity": [19, 15, 150],
                                   "location": ["tools", "tools", "hardware"]})

# alice bought a hammer and a wrench on 1/2/2025,
# bob bought screws on 3/4/2025, and joe bought a hammer on 5/6/2025.
# Note that Pandas has much better representations for dates/times, but they
# are beyond our scope and have been simplified to strings
hardware_purchases = pd.DataFrame({"customer_name": ["alice", "alice", "bob", "joe"],
                                   "product": ["hammer", "wrench", "screws (20)", "hammer"],
                                   "when": ["1/2/2025", "1/2/2025", "3/4/2025", "5/6/2025"]})

print(pd.concat([hardware_inventory, hardware_purchases], ignore_index=True))

       product  price  quantity  location customer_name      when
0       hammer   20.0      19.0     tools           NaN       NaN
1       wrench   30.0      15.0     tools           NaN       NaN
2  screws (20)    2.5     150.0  hardware           NaN       NaN
3       hammer    NaN       NaN       NaN         alice  1/2/2025
4       wrench    NaN       NaN       NaN         alice  1/2/2025
5  screws (20)    NaN       NaN       NaN           bob  3/4/2025
6       hammer    NaN       NaN       NaN           joe  5/6/2025


If you run the cell above, you'll see a whole lot of `NaN`s appearing in the data.
The columns of the concatenated table are based on the set union of the columns from the original data, meaning every row now contains columns from both data sets.
However, depending on which data set the row came from, there will always be some columns that don't have values for that row, and `NaN` gets used in these places.
While the `DataFrame` objects have technically been combined, they have not been meaningfully cross-referenced.

This sort of cross-referencing is best done instead by the `merge` method on `DataFrame` objects.
The `merge` method takes another `DataFrame` object.
By default, merge will perform this cross-referencing with any overlapping columns, though it's usually a good idea to manually specify which column to perform the cross-referencing with by passing the optional `on` keyword parameter.
This is shown in the cell below:

In [8]:
cross_referenced = hardware_inventory.merge(hardware_purchases, on="product")
print(cross_referenced)

       product  price  quantity  location customer_name      when
0       hammer   20.0        19     tools         alice  1/2/2025
1       hammer   20.0        19     tools           joe  5/6/2025
2       wrench   30.0        15     tools         alice  1/2/2025
3  screws (20)    2.5       150  hardware           bob  3/4/2025


If you run the cell above, you'll see that you have all the same columns as from the call to `concat`.
However, the number of rows now corresponds to the number of rows now corresponds to the number of rows in `hardware_purchases`.
There is no information about price, quantity, and location in the `hardware_purchases` `DataFrame`.
For those missing values, `merge` consulted the `hardware_inventory` table, specifically looking for the row information corresponding to a value with the same column value as `"product"`.
For example, the `"price"` column for row `0` was determined by taking the `"price"` information for `"hammer"` in `hardware_inventory`.
Importantly, both `hardware_inventory` and `hardware_purchases` have a `"product"` column containing the same sorts of values, allowing for this cross-referencing to even be possible.

Now that we have this cross-referenced `DataFrame`, we can much more easily perform analyses that require information from both tables.
For example, to determine the total value of all of Alice's purchases, we can now do:

In [9]:
print(cross_referenced[cross_referenced["customer_name"] == "alice"]["price"].sum())

50.0


This would not have been possible to do so concisely if we had to work with multiple `DataFrame` tables, and likely would have necessitated a loop (with all the extra code and performance drawbacks).

The sort of cross-referencing shown is known as a _join_, borrowing terminology from [joins in Structured Query Language (SQL)](https://en.wikipedia.org/wiki/Join_(SQL)#Inner_join_and_NULL_values).
To explain this terminology, the idea is that we can view the values within whatever column we are joining on as a set.
Because we have two columns in play with the same sorts of values (one `"product"` column from `hardware_inventory`, and another `"product"` column from `hardware_purchases`), we have two sets in play.
We can visually represent this as a Venn diagram, shown below:

![venn1](venn1.png)

There are a number of different kinds of joins, which all refer to a two-set Venn diagram, same as the above.
The distinction between these joins is based on what parts of the diagram we are attempting to show in the result.
For example, we specifically performed an _inner join_ when we created `cross_referenced` above, which is the default of the `merge` method.
An inner join requests the intersection between the two parts of the Venn diagram, that is, the inner overlapping part of the diagram.
Relevant to Pandas, the `merge` method also supports (among others):

- Left joins, which only use the values from the leftmost `DataFrame`.  In the Venn diagram, this is everything in the left circle.
- Right joins, which only use the values from the rightmost `DataFrame`.  In the Venn diagram, this is everything in the right circle.
- Outer joins, which use all the values from both `DataFrame`s.  In the Venn diagram, this is everything.

In this specific example, since all keys involved are in the intersection, all these different kinds of joins will result in the same row data, though the ordering of the rows will differ based on the join type.
In the next step we will look at some of these other joins, but for now we will only consider inner joins.

### Try this Yourself ###

In the next cell, `DataFrame` objects for `patients` and `blood_pressures` have been defined.
Unlike the prior data, this format separates the age of patients into the `patients` `DataFrame`.
Individual blood pressure readings are in the `blood_pressures` table, which now has multiple readings for different patients over time.

You need to do two things in this cell:

1. Using `merge` with an inner join, cross-reference these two `DataFrame`s into a single `DataFrame`, based on the patient name.
2. On this joined `DataFrame`, compute the average blood pressure reading across all readings for patients under 50 years old.  Print out the result of this.

In [13]:
patients = pd.DataFrame({"name": ["alex", "bob", "megan", "john", "mary", "maggie", "sam"],
                         "age": [45, 57, 42, 62, 48, 27, 21]})
blood_pressures = pd.DataFrame({"name": ["alex", "bob", "bob", "megan", "megan", "megan", "john",
                                         "john", "mary", "mary", "mary", "maggie", "sam", "sam"],
                                "blood_pressure": [135, 122, 118, 121, 132, 135, 126,
                                                   132, 124, 120, 119, 136, 127, 139],
                                "when": ["1/2/2025", "1/3/2020", "1/7/2023", "2/4/2020", "5/6/2023", "8/2/2024", "1/2/2024",
                                         "1/3/2025", "1/4/2020", "1/8/2023", "6/2/2024", "1/3/2020", "2/4/2020", "5/6/2023"]})

# Write your statement here which will do an inner join of patients and blood_pressures
# You should then bind this to the variable combined_patients
combined_patients = patients.merge(blood_pressures, on="name")

print(combined_patients)
# above statement should print:
#       name  age  blood_pressure      when
# 0     alex   45             135  1/2/2025
# 1      bob   57             122  1/3/2020
# 2      bob   57             118  1/7/2023
# 3    megan   42             121  2/4/2020
# 4    megan   42             132  5/6/2023
# 5    megan   42             135  8/2/2024
# 6     john   62             126  1/2/2024
# 7     john   62             132  1/3/2025
# 8     mary   48             124  1/4/2020
# 9     mary   48             120  1/8/2023
# 10    mary   48             119  6/2/2024
# 11  maggie   27             136  1/3/2020
# 12     sam   21             127  2/4/2020
# 13     sam   21             139  5/6/2023

# Below, write a single expression which, using Pandas, will compute the average
# blood pressure reading across all measurements for patients less than 50, using
# your combined_patients DataFrame from before.
# Be sure to print out the result.
# This should print 128.8
print()
print(combined_patients[combined_patients["age"] < 50]["blood_pressure"].mean())

      name  age  blood_pressure      when
0     alex   45             135  1/2/2025
1      bob   57             122  1/3/2020
2      bob   57             118  1/7/2023
3    megan   42             121  2/4/2020
4    megan   42             132  5/6/2023
5    megan   42             135  8/2/2024
6     john   62             126  1/2/2024
7     john   62             132  1/3/2025
8     mary   48             124  1/4/2020
9     mary   48             120  1/8/2023
10    mary   48             119  6/2/2024
11  maggie   27             136  1/3/2020
12     sam   21             127  2/4/2020
13     sam   21             139  5/6/2023

128.8


## Step 3: Use `merge` To Combine two Pandas `DataFrame` Objects via an Outer Join ##

### Background: Outer Joins ###

An outer join considers all values between both columns specified in the join, as opposed to only those values which are present in both.
In set terms, an inner join (shown previously) only considers values which intersect between the two `DataFrame` objects, whereas an outer join considers all.

When calling `merge`, we can optionally set the kind of merge performed by passing a string as a keyword argument `how`.
This string defaults to `"inner"` (hence a default of an inner join), but we can also use `"outer"` for an outer join; the [official Pandas documentation shows all the different options for `how`](https://pandas.pydata.org/docs/reference/api/pandas.merge.html#pandas.merge).
This is all illustrated in the next cell, which is an updated version of the hardware inventory from before.
In this case, `new_hardware_inventory` adds a new item: `"jigsaw"`.
Importantly, none of the rows in `hardware_purchases` mention `"jigsaw"`.
Both an inner join and outer join is performed side-by-side, so we can see the differences.

In [14]:
new_hardware_inventory = pd.DataFrame({"product": ["hammer", "wrench", "screws (20)", "jigsaw"],
                                       "price": [20, 30, 2.5, 40],
                                       "quantity": [19, 15, 150, 12],
                                       "location": ["tools", "tools", "hardware", "tools"]})
# copied for convenience
hardware_purchases = pd.DataFrame({"customer_name": ["alice", "alice", "bob", "joe"],
                                   "product": ["hammer", "wrench", "screws (20)", "hammer"],
                                   "when": ["1/2/2025", "1/2/2025", "3/4/2025", "5/6/2025"]})
inner = new_hardware_inventory.merge(hardware_purchases, on="product", how="inner")
print(inner)

print()
outer = new_hardware_inventory.merge(hardware_purchases, on="product", how="outer")
print(outer)

       product  price  quantity  location customer_name      when
0       hammer   20.0        19     tools         alice  1/2/2025
1       hammer   20.0        19     tools           joe  5/6/2025
2       wrench   30.0        15     tools         alice  1/2/2025
3  screws (20)    2.5       150  hardware           bob  3/4/2025

       product  price  quantity  location customer_name      when
0       hammer   20.0        19     tools         alice  1/2/2025
1       hammer   20.0        19     tools           joe  5/6/2025
2       jigsaw   40.0        12     tools           NaN       NaN
3  screws (20)    2.5       150  hardware           bob  3/4/2025
4       wrench   30.0        15     tools         alice  1/2/2025


As shown, with the inner join, we don't have any rows containing `"jigsaw"`.
However, we _do_ have a row containing `"jigsaw"` for the outer join.
With the outer join, because we don't have corresponding information for `"customer_name"` or `"when"`, those values are filled-in with `NaN`.

Graphically, this distinction between an inner join and an outer join for this example is shown below.
![venn2](venn2.png)

The inner join will only get the items within the inside (overlapping) part of the diagram.
However, the outer join will get everything, including `"jigsaw"`.

The introduction of `NaN`s may make outer joins seem useless, because this is effectively fake data that is introduced into the data set.
Indeed, every non-inner join suffers from the same problem, because fundamentally in these cases, we don't have corresponding data for a given column value.
There is a reason why inner join is the default; this usually makes the most sense.
While some care needs to be taken with other kinds of joins, this can nonetheless still be useful, depending on what you're doing.
For example, let's do a slightly modified version, of this, where:

- `new_hardware` is intended only to represent _current_ inventory.
- `new_purchases` (a new variable) represents _all_ purchases historically made, including those of items which are no longer kept in stock.

This is shown in the next cell.

In [15]:
# copied for convenience
new_hardware_inventory = pd.DataFrame({"product": ["hammer", "wrench", "screws (20)", "jigsaw"],
                                       "price": [20, 30, 2.5, 40],
                                       "quantity": [19, 15, 150, 12],
                                       "location": ["tools", "tools", "hardware", "tools"]})
new_hardware_purchases = pd.DataFrame({"customer_name": ["alice", "alice", "bob", "joe", "carl"],
                                       "product": ["hammer", "wrench", "screws (20)", "hammer", "table saw"],
                                       "when": ["1/2/2025", "1/2/2025", "3/4/2025", "5/6/2025", "5/12/2015"]})

new_combined_hardware = new_hardware_inventory.merge(new_hardware_purchases, on="product", how="outer")
print(new_combined_hardware)

       product  price  quantity  location customer_name       when
0       hammer   20.0      19.0     tools         alice   1/2/2025
1       hammer   20.0      19.0     tools           joe   5/6/2025
2       jigsaw   40.0      12.0     tools           NaN        NaN
3  screws (20)    2.5     150.0  hardware           bob   3/4/2025
4    table saw    NaN       NaN       NaN          carl  5/12/2015
5       wrench   30.0      15.0     tools         alice   1/2/2025


As shown, table saws are no longer kept in inventory, but one was sold back in 2015.
The `new_combined_hardware` table resulted from an outer join, and some `NaN`s are in the table.
However, if we were interested in identifying all products ever available for purchase, this data is perfectly fine, as shown in the next cell:

In [16]:
# The unique() method is on Series objects, and gives back a
# NumPy array of all the unique items in a Series
print(new_combined_hardware["product"].unique())

['hammer' 'jigsaw' 'screws (20)' 'table saw' 'wrench']


In fact, not only is `new_combined_hardware` ok for this analysis, neither `new_hardware_inventory` nor `new_hardware_purchases` is ok alone for this same analysis.
Specifically, `new_hardware_inventory` doesn't contain table saws, since they are no longer sold; only looking at products in `new_hardware_purchases` would miss anything that is no longer sold.
On the flip side, `new_hardware_purchases` only details items which have actually been sold; only looking at `new_hardware_purchases` would miss items like jigsaws, which are available for purchase, but no one has bought one yet.

### Try this Yourself ###

The next cell defines two `DataFrame` objects named `first` and `second`, which share a column named `"id"`.
Perform an outer join of these two objects, and print out the resulting joined `DataFrame` object.

In [19]:
first = pd.DataFrame({"id": ["foo", "bar", "baz"],
                      "first_value": [3, 2, 7]})
second = pd.DataFrame({"id": ["bar", "baz", "blah"],
                       "second_value": [2.3, 4.1, 8.9]})

# Call the merge method to perform an outer join using column id
# Print out the resulting merged DataFrame object
df = first.merge(second, on="id", how="outer")
print(df)

# This should print the following:
#      id  first_value  second_value
# 0   bar          2.0           2.3
# 1   baz          7.0           4.1
# 2  blah          NaN           8.9
# 3   foo          3.0           NaN

     id  first_value  second_value
0   bar          2.0           2.3
1   baz          7.0           4.1
2  blah          NaN           8.9
3   foo          3.0           NaN


## Step 4: Submit via Canvas ##

Be sure to **save your work**, then log into [Canvas](https://canvas.csun.edu/).  Go to the COMP 502 course, and click "Assignments" on the left pane.  From there, click "Assignment 28".  From there, you can upload your `28_pandas_combining_dataframes.ipynb` file.

You can turn in the assignment multiple times, but only the last version you submitted will be graded.

### Special Thanks to Dr. Glenn Bruns ###

Special thanks to [Dr. Glenn Bruns](https://csumb.edu/scd/glenn-bruns/) at California State University, Monterey Bay, for providing me with closely-related materials which were used in the creation of this assignment.